In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from scipy import stats
import commands

In [16]:
def get_even_num(num=93):
    runs = []
    evens = []
    j = 0
    for i in range(num):
        if (i != 0) & (i != 2):
            if i % 2 == 0:
                evens.append(str(i))
                j += 1
                if j == 5: 
                    runs.append(evens)
                    evens = []
                    j = 0
    return runs

In [13]:
def get_minus_num(base_runs):
    li = []
    for i in range(len(base_runs)):
        li.append([[base_runs[i][1],base_runs[i][0]], [base_runs[i][3],base_runs[i][0]], [base_runs[i][1],base_runs[i][3]]])
    return li

In [83]:
data_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/"
img_path = data_path+"processed/NIFTI/"
save_path = data_path+"raw/"

sub_path_36 = data_path+"external/R21_36_Cyl_Aqueous_v3_simplot/"
sub_path_42 = data_path+"external/R21_42_SphereProto3/"
sub_path_43 = data_path+"external/R21_43_Sphere_Proto3/"
sub_path_44 = data_path+"external/R21_44_Sphere_Proto3/"
sub_path_45 = data_path+"external/R21_45_Sphere_Proto3/"
# tes = [2.46, 4.61, 7.09, 10.5, 15.0, 20.0, 24.5, 29.0, 33.5, 38.0, 42.5, 47.0]

base_runs = get_even_num()
bp = get_minus_num(base_runs)

In [72]:
br = [['66', '68', '70', '72', '74'],
 ['76', '78', '80', '82', '84'],
 ['86', '88', '90', '92', '94']]
bp = get_minus_num(br)

In [76]:
br = base_runs[3:6]
bp = get_minus_num(br)

In [77]:
bp

[[['36', '34'], ['40', '34'], ['36', '40']],
 [['46', '44'], ['50', '44'], ['46', '50']],
 [['56', '54'], ['60', '54'], ['56', '60']]]

In [30]:
# DOESN'T WORK
def bet_t1(sub_path, base_fldr):
    cwd_path = sub_path+"bmaps/"
    tdir = cwd_path+base_fldr+"/"
    t1dir = cwd_path+"T1/"
    
    print commands.getoutput("bet "+t1dir+"/T1.nii T1_bet -Z -m -s -A")
#     print commands.getoutput("3dresample -master "+tdir+"4d_fmap_Mean.nii.gz -prefix "+t1dir+"T1_bet_mask_4mm_iso.nii.gz -input "+t1dir+"T1_bet_mask.nii.gz")
    

In [ ]:
#42
bet_t1(sub_path_42, '68minus66')

In [31]:
#43
bet_t1(sub_path_43, '36minus34')

In [ ]:
#44
bet_t1(sub_path_44, '36minus34')

In [95]:
def upscale_fmap_mean(sub_path, base_runs, bp, mins):
    for n in range(len(base_runs)):
        cwd_path = sub_path+"bmaps/"
#         cwd_path = sub_path+"NIFTI/bmaps_numpy/"
        for m in range(2):
            tdir = cwd_path+bp[n][m][0]+"minus"+bp[n][m][1]+"/"
            t1dir = cwd_path+"T1/"
            print commands.getoutput("3dBlurInMask -input "+tdir+"4d_fmap_Mean.nii.gz -FWHM 8 -prefix "+tdir+"4d_fmap_Mean_blur_bet_mask.nii.gz -mask "+t1dir+"T1_bet_mask_4mm_iso.nii.gz")
            print commands.getoutput("3dresample -input "+tdir+"4d_fmap_Mean_blur_bet_mask.nii.gz -master "+t1dir+"T1.nii -rmode Li -prefix "+tdir+"4d_fmap_Mean_blur_bet_mask_1mm_Li.nii.gz")
            print commands.getoutput("fslmaths "+tdir+"4d_fmap_Mean_blur_bet_mask_1mm_Li.nii.gz -mul "+t1dir+"T1_bet_mask.nii.gz "+tdir+"4d_fmap_Mean_final_result.nii")
            print commands.getoutput("fslroi "+tdir+"4d_fmap_Mean_final_result.nii.gz "+tdir+"4d_fmap_Mean_final_result_fslroi.nii "+mins[0]+" 73 "+mins[1]+" 73 "+mins[2]+" 173")
            

In [93]:
def upscale_bval(sub_path, base_runs, bp, mins):
    for n in range(len(base_runs)):
        cwd_path = sub_path+"bmaps/"
#         cwd_path = sub_path+"NIFTI/bmaps_numpy/"
        for m in range(2):
            tdir = cwd_path+bp[n][m][0]+"minus"+bp[n][m][1]+"/"
            t1dir = cwd_path+"T1/"
            print commands.getoutput("3dBlurInMask -input "+tdir+"Bvals.nii -FWHM 8 -prefix "+tdir+"Bvals_blur_bet_mask.nii.gz -mask "+t1dir+"T1_bet_mask_4mm_iso.nii.gz")
            print commands.getoutput("3dresample -input "+tdir+"Bvals_blur_bet_mask.nii.gz -master "+t1dir+"T1.nii -rmode Li -prefix "+tdir+"Bvals__blur_bet_mask_1mm_Li.nii.gz")
            print commands.getoutput("fslmaths "+tdir+"Bvals__blur_bet_mask_1mm_Li.nii.gz -mul "+t1dir+"T1_bet_mask.nii.gz "+tdir+"Bvals_final_result.nii")
            print commands.getoutput("fslroi "+tdir+"Bvals_final_result.nii.gz "+tdir+"Bvals_final_result_fslroi.nii "+mins[0]+" 73 "+mins[1]+" 73 "+mins[2]+" 173")
            

In [89]:
# detrend
def upscale_fmap(sub_path, base_runs, bp, mins):
    for n in range(len(base_runs)):
        cwd_path = sub_path+"bmaps/"
#         cwd_path = sub_path+"NIFTI/bmaps_numpy/"
        for m in range(2):
            tdir = cwd_path+bp[n][m][0]+"minus"+bp[n][m][1]+"/"
            t1dir = cwd_path+"T1/"
#             print commands.getoutput("3dBlurInMask -input "+tdir+"detrend14d_fmap_Mean.nii.gz -FWHM 8 -prefix "+tdir+"detrend14d_fmap_Mean_blur_bet_mask.nii.gz -mask "+t1dir+"T1_bet_mask_4mm_iso.nii.gz")
#             print commands.getoutput("3dBlurInMask -input "+tdir+"detrend1.nii.gz -FWHM 8 -prefix "+tdir+"detrend14d_fmap_Mean_blur_bet_mask.nii.gz -mask "+t1dir+"T1_bet_mask_4mm_iso.nii.gz")
#             print commands.getoutput("3dresample -input "+tdir+"detrend14d_fmap_Mean_blur_bet_mask.nii.gz -master "+t1dir+"T1.nii -rmode Li -prefix "+tdir+"detrend14d_fmap_Mean_blur_bet_mask_1mm_Li.nii.gz")
#             print commands.getoutput("fslmaths "+tdir+"detrend14d_fmap_Mean_blur_bet_mask_1mm_Li.nii.gz -mul "+t1dir+"T1_bet_mask.nii.gz "+tdir+"final_result.nii")
#             print commands.getoutput("fslroi "+tdir+"final_result.nii.gz "+tdir+"final_result_fslroi.nii "+mins[0]+" 77 "+mins[1]+" 77 "+mins[2]+" 77")
            print commands.getoutput("fslroi "+tdir+"final_result.nii.gz "+tdir+"final_result_fslroi.nii "+mins[0]+" 73 "+mins[1]+" 73 "+mins[2]+" 173")
            

In [96]:
# 36 
br = base_runs[5:6]
bp = get_minus_num(br)
mins = ['66', '103', '43']
# upscale_fmap(sub_path_36, br, bp, mins)
# upscale_bval(sub_path_36, br, bp, mins)
upscale_fmap_mean(sub_path_36, br, bp, mins)

++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 38068
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous_v3_simplot/bmaps/56minus54/4d_fmap_Mean_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 38068
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_36_Cyl_Aqueous_v3_simplot/bmaps/60minus54/4d_fmap_Mean_blur_bet_mask.nii.gz





In [82]:
bp

[[['56', '54'], ['60', '54'], ['56', '60']]]

In [78]:
base_runs

[['4', '6', '8', '10', '12'],
 ['14', '16', '18', '20', '22'],
 ['24', '26', '28', '30', '32'],
 ['34', '36', '38', '40', '42'],
 ['44', '46', '48', '50', '52'],
 ['54', '56', '58', '60', '62'],
 ['64', '66', '68', '70', '72'],
 ['74', '76', '78', '80', '82'],
 ['84', '86', '88', '90', '92']]

In [41]:
# 42
br = [['66', '68', '70', '72', '74'],
 ['76', '78', '80', '82', '84'],
 ['86', '88', '90', '92', '94']]
bp = get_minus_num(br)
mins = ['61', '71', '87']
upscale_fmap(sub_path_42, br, bp, mins)

In [49]:
# 43
br = base_runs[3:5]
bp = get_minus_num(br)
mins = ['62', '107', '87']
upscale_fmap(sub_path_43, br, bp, mins)

In [56]:
# 44
br = base_runs[3:6]
bp = get_minus_num(br)
mins = ['62', '107', '87']
upscale_fmap(sub_path_44, br, bp, mins)

In [57]:
def roi_roast(sub_path, fldr):
    cwd_path = sub_path+"bmaps/"
    t1dir = cwd_path+"T1/"
    for m in range(2):
        roast_path = sub_path+"roast/"+fldr[m]+"/"
        print commands.getoutput("fslmaths "+roast_path+"Bfieldz_wJ.nii -mul "+t1dir+"T1_bet_mask.nii.gz "+roast_path+"Bfieldz_masked.nii")
        print commands.getoutput("fslmaths "+roast_path+"Bfieldz_masked.nii.gz -mul 1000000000 "+roast_path+"Bfieldz_masked_nT.nii")
        print commands.getoutput("fslroi "+roast_path+"Bfieldz_masked_nT.nii "+roast_path+"Bfieldz_masked_nT_fslroi.nii "+mins[0]+" 77 "+mins[1]+" 77 "+mins[2]+" 77")


In [ ]:
#42
fldr = ['pos', 'neg']
roi_roast(sub_path_42, fldr)

In [58]:
#43
fldr = ['pos', 'neg']
roi_roast(sub_path_43, fldr)

In [ ]:
# #44
# fldr = ['pos', 'neg']
# roi_roast(sub_path_44, fldr)

In [99]:
def upscale_predict(sub_path, files, mins):
    cwd_path = sub_path+"bmaps/"
    t1dir = cwd_path+"T1/"
    
    for n in range(len(files)):
        fl = files[n].replace('.nii.gz', '')
        out = fl.replace("nn_nonzero_best_cly_", "")
        print commands.getoutput("3dBlurInMask -input "+img_path+files[n]+" -FWHM 8 -prefix "+img_path+"upscaled/"+fl+"_blur_bet_mask.nii.gz -mask "+cwd_path+"6minus4/detrend1_fslroi.nii.gz")
#         print commands.getoutput("3dBlurInMask -input "+img_path+files[n]+" -FWHM 8 -prefix "+img_path+"upscaled/"+fl+"_blur_bet_mask.nii.gz -mask "+t1dir+"T1_bet_mask_4mm_iso.nii.gz")
        print commands.getoutput("3dresample -input "+img_path+"upscaled/"+fl+"_blur_bet_mask.nii.gz -master "+t1dir+"T1.nii -rmode Li -prefix "+img_path+"upscaled/"+fl+"_blur_bet_mask_1mm_Li.nii.gz")
        print commands.getoutput("fslmaths "+img_path+"upscaled/"+fl+"_blur_bet_mask_1mm_Li.nii.gz -mul "+t1dir+"T1_bet_mask.nii.gz "+img_path+"upscaled/"+"final_result_"+out+".nii")
#         print commands.getoutput("fslroi "+img_path+"upscaled/"+"final_result_"+out+".nii " +img_path+"upscaled/"+"final_result_"+out+"_fslroi.nii "+mins[0]+" 77 "+mins[1]+" 77 "+mins[2]+" 77")
        print commands.getoutput("fslroi "+img_path+"upscaled/"+"final_result_"+out+".nii " +img_path+"upscaled/"+"final_result_"+out+"_fslroi.nii "+mins[0]+" 73 "+mins[1]+" 73 "+mins[2]+" 173")
        

In [100]:
# 36
nifti = [
'nn_optimized_32to38_neg.nii.gz',
'nn_optimized_32to38_pos.nii.gz']
mins = ['66', '103', '43']
upscale_predict(sub_path_36, nifti, mins)

++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 15884
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_optimized_32to38_neg_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 15884
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_optimized_32to38_pos_blur_bet_mask.nii.gz





In [69]:
# 42
nifti = [
'nn_nonzero_best_cly_42_1_neg.nii.gz',
'nn_nonzero_best_cly_42_1_pos.nii.gz',
'nn_nonzero_best_cly_42_2_neg.nii.gz',
'nn_nonzero_best_cly_42_2_pos.nii.gz',
'nn_nonzero_best_cly_42_3_neg.nii.gz',
'nn_nonzero_best_cly_42_3_pos.nii.gz']
mins = ['61', '71', '87']
upscale_predict(sub_path_42, nifti, mins)

++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28570
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_42_1_neg_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28570
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_42_1_pos_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28570
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
** E

In [70]:
# 43
nifti = [
'nn_nonzero_best_cly_43_1_neg.nii.gz',
'nn_nonzero_best_cly_43_1_pos.nii.gz',
'nn_nonzero_best_cly_43_2_neg.nii.gz',
'nn_nonzero_best_cly_43_2_pos.nii.gz']
mins = ['62', '107', '87']
upscale_predict(sub_path_43, nifti, mins)

++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28002
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_43_1_neg_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28002
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_43_1_pos_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28002
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ O

In [71]:
# 44
nifti = [
'nn_nonzero_best_cly_44_1_neg.nii.gz',
'nn_nonzero_best_cly_44_1_pos.nii.gz',
'nn_nonzero_best_cly_44_2_neg.nii.gz',
'nn_nonzero_best_cly_44_2_pos.nii.gz',
'nn_nonzero_best_cly_44_3_neg.nii.gz',
'nn_nonzero_best_cly_44_3_pos.nii.gz']
mins = ['62', '107', '87']
upscale_predict(sub_path_44, nifti, mins)

++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28077
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_44_1_neg_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28077
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ Output dataset /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/NIFTI/upscaled/nn_nonzero_best_cly_44_1_pos_blur_bet_mask.nii.gz



++ 3dBlurInMask: AFNI version=AFNI_22.1.14 (Jun 24 2022) [64-bit]
++ Authored by: RW Cox
++ Number of voxels in mask = 28077
++ forcing output to be stored in float format
++ mri_blur3D: #iter=6 fx=0.03006 fy=0.03006 fz=0.03006
++ O